## Question 0

In [1]:
import pandas as pd # conventional alias
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

salary = pd.read_csv("http://www.webpages.uidaho.edu/~stevel/Datasets/salary_uk.csv")
salary.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


# Data Prepare

In [ ]:
#Raw variables will be dropped; Id were unique variable for each person, so as full description, they will also be dropped.

In [2]:
salary2=salary.drop(columns=['Id','FullDescription','SalaryRaw','LocationRaw'])
#Dummy code for variables "LocationNormalized","Category","SourceName".
salary2=pd.get_dummies(salary2, columns=["LocationNormalized", "Category","SourceName"], prefix=["Loc", "Cat","Source"])
#Fix missing values
salary2 = salary2.fillna(salary2['ContractType'].value_counts().index[0])
salary2 = salary2.fillna(salary2['Title'].value_counts().index[0])
salary2 = salary2.fillna(salary2['ContractTime'].value_counts().index[0])
#Dummy code for variables "ContractType", "ContractTime","Title","Company".
salary2=pd.get_dummies(salary2, columns=["ContractType", "ContractTime","Title","Company"])

In [3]:
#Combine two dumming dataframes
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer(stop_words='english', max_features=10000)
x=vect.fit_transform(salary.FullDescription)
fulldesc=pd.DataFrame(x.toarray(), columns=vect.get_feature_names())
salary3=pd.concat([salary2,fulldesc], axis=1)
salary3.head()

C:\Users\Summer\Anaconda3\envs\py27\lib\site-packages\pandas\core\indexes\base.py:2022: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine


,SalaryNormalized,Loc_Abbots Langley,Loc_Aberdare,Loc_Aberdeen,Loc_Aberdeenshire,Loc_Abertillery,Loc_Abingdon,Loc_Aboyne,Loc_Accrington,Loc_Adlington,...,youtube,youview,yr,yrs,zahid,zealand,zend,zero,zone,zuken
0,25000,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27500,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25000,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Analysis

### Preprocessing

In [5]:
from sklearn.model_selection import train_test_split

x=salary3.drop('SalaryNormalized',axis=1)
y=salary3.SalaryNormalized
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

### Linear regression model

In [6]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(x_train, y_train)
print("Training set score: {:.2f}".format(lr.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(x_test, y_test)))

Training set score: 1.00
Test set score: -14067753898746947584.00


### K Nearest Neighbor

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
print("Training set score: {:.2f}".format(knn.score(x_train, y_train)))
print("Test set score: {:.2f}".format(knn.score(x_test, y_test)))

Training set score: 0.36
Test set score: 0.11


In [15]:
y_pred_class = knn.predict(x_test)
print metrics.accuracy_score(y_test, y_pred_class)

0.1132


### Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
print("Training set score: {:.2f}".format(tree.score(x_train, y_train)))
print("Test set score: {:.2f}".format(tree.score(x_test, y_test)))

Training set score: 1.00
Test set score: 0.15


In [14]:
y_pred_class = tree.predict(x_test)
print metrics.accuracy_score(y_test, y_pred_class)

0.1484


### Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

rnd = RandomForestClassifier()
rnd.fit(x_train,y_train)
print("Training set score: {:.2f}".format(rnd.score(x_train, y_train)))
print("Test set score: {:.2f}".format(rnd.score(x_test, y_test)))

Training set score: 0.99
Test set score: 0.16


In [19]:
y_pred_class = rnd.predict(x_test)
print metrics.accuracy_score(y_test, y_pred_class)

0.1636


### Ridge Regression

In [22]:
from sklearn.linear_model import Ridge

ridge = Ridge().fit(x_train, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test, y_test)))

Training set score: 0.95
Test set score: 0.64


### Lasso

In [26]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(x_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(x_test, y_test)))

Training set score: 0.92
Test set score: 0.62


### Conclusion

Overall Ridge Regression had the highest trainning score and test score. And it's pretty fast to get the final result while linear regression cost lots of time.